Om de cel hier onder te runnen moet er een package geinstalleerd worden via anaconda prompt. Deze package zorgt ervoor dat we functies uit andere scripts kunnen gerbuiken in dit script.
<break>
Tik het volgende in anaconda prompt:
<br \>
**pip install ipynb**

In [1]:
%%capture
import pandas as pd
import numpy as np
from numpy.linalg import inv
from numpy.linalg import det
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
from ipynb.fs.full.Dataset import getdata
from ipynb.fs.full.Dataset import getreturns
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
from scipy.special import gammaln
#import scipy
from scipy.special import gamma
from scipy import integrate
from scipy.special import kv as kv

###### Hier onder wordt de dataset verkregen. Ik gebruik 2010 tot en met 2015 als mijn in-sample data.
<break>
Hiervoor gebruik een functie die in een ander script (Dataset) is geschreven

In [2]:
df = getreturns()
mInSampleReturns = df.loc[:"2015"]
iT=len(mInSampleReturns)

Hier schat ik de garch modellen met skewed student-t verdeling

In [3]:
def Parameters_GARCH_Skew(vReturns):
    dOmega_ini=0.1
    dAlpha_ini=0.1
    dBeta_ini=0.8
    dNu_ini=6
    dXi=1
    
    vTheta=np.array([dOmega_ini, dAlpha_ini, dBeta_ini, dNu_ini, dXi])
    def Log_likelihood_GARCH_Skew(vTheta,vReturns):
        iT=len(vReturns)
        dOmega=vTheta[0]
        dAlpha=vTheta[1]
        dBeta=vTheta[2]
        dNu=vTheta[3]
        dXi=vTheta[4]
        vH=np.zeros(iT)
        
        for t in range(iT):
            if t == 0:
                vH[t] = np.var(vReturns) 
            else:
                vH[t] = dOmega + dAlpha*vReturns[t-1]**2 + dBeta * vH[t-1] 
        
        dM=gamma( (dNu-1)/2 ) / ( gamma(dNu/2) )*np.sqrt( (dNu-2)/np.pi )*( dXi-1/dXi )
        dS=np.sqrt((dXi**2+(1/dXi**2)-1)-dM**2)
        vI = (dS * (vReturns/np.sqrt(vH))) + dM
        for i in range(len(vI)):
            if vI[i]<0:
                vI[i]=-1
            else:
                vI[i]=1
                      
        vLogPdf = gammaln( (dNu+1)/2 ) - gammaln( dNu/2 ) - 1/2*np.log( ( dNu-2 ) *np.pi ) -1/2 * np.log(vH) \
              + np.log( dS ) + np.log( 2 / ( dXi+1/dXi ) ) - ( ( dNu+1 )/2 )*np.log( 1 + ( (dS * ( ( vReturns/np.sqrt( vH ) ) ) + dM)**2/(dNu-2))*dXi**(-2*vI)) 
        
        
        return -np.sum(vLogPdf)
    
    def Optimizer(vReturns, initials, function, bnds):
        result = minimize(function, initials, args=(vReturns), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    bounds = ((0, 1), (0, 1), (0, 1), (2.1, 50), (0.00001,100))
    result=Optimizer(vReturns, vTheta, Log_likelihood_GARCH_Skew, bounds)
    return result.x, -result.fun, result.success

In [4]:
(parameter1_Skew,likelihood1_Skew,success1_Skew)=Parameters_GARCH_Skew(np.array(mInSampleReturns.iloc[:,0]))
print(parameter1_Skew)
print(likelihood1_Skew)
print(success1_Skew)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2489.657289021924
            Iterations: 16
            Function evaluations: 129
            Gradient evaluations: 16
[ 0.03851373  0.0887035   0.89393724  6.87100222  0.90919401]
-2489.657289021924
True


In [5]:
(parameter2_Skew,likelihood2_Skew,success2_Skew)=Parameters_GARCH_Skew(np.array(mInSampleReturns.iloc[:,1]))
print(parameter2_Skew)
print(likelihood2_Skew)
print(success2_Skew)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2213.559393308958
            Iterations: 15
            Function evaluations: 117
            Gradient evaluations: 15
[ 0.0234867   0.09196745  0.89452381  7.07932222  0.90333946]
-2213.559393308958
True


#### Hier schat ik de waardes van de sigma's over tijd voor alle 2 de tijdreeksen

In [6]:
def ComputeSigma2GARCH(vTheta1,vTheta2,mReturns):
    iT = mReturns.shape[1]
    iDimension = mReturns.shape[0]
    mH = np.zeros((iDimension,iT))
    dOmega1 = vTheta1[0]
    dOmega2 = vTheta2[0]
    dAlpha1 = vTheta1[1]
    dAlpha2 = vTheta2[1]
    dBeta1 = vTheta1[2]
    dBeta2 = vTheta2[2]
    for t in range(iT):
        if t==0:
            mH[0,t]=np.var(mReturns[0,:])
            mH[1,t]=np.var(mReturns[1,:])
        else:
            mH[0,t]=dOmega1 + dAlpha1 * mReturns[0,t-1]**2 + dBeta1 * mH[0,t-1]
            mH[1,t]=dOmega2 + dAlpha2 * mReturns[1,t-1]**2 + dBeta2 * mH[1,t-1]
    return mH

In [7]:
mSigma2=ComputeSigma2GARCH(parameter1_Skew,parameter2_Skew,np.array(mInSampleReturns).T)

## Copula part

Vanaf hieronder zal ik de copula LL gaan uitwerken. Eerst moet ik de PIT transformatie toepassen op de returns. Hierbij moet ik de kans berekenen dat:
$$ P(Y_t<y_t)=u_t \quad Y_t\sim \text{Skew}(0,\sigma_t^2, \nu, \xi) $$
Waarbij $Y_t$ een stochast is en $y_t$ de realisatie.
Er is geen ingebouwde functie voor de cdf van generalized skewed student-t verdeling. Dus moet dat eerst gehardcode worden.

### Probabability integral transformation (PIT)
Hier bereken ik de pdf waarde van alle gridpunten met de variance van een bepaald tijdstip t. Wat ik in deze functie doe is de verdeling van elk tijdstip t op te stellen met de bijbehorende variance op tijdstip t. Als de pdf is bepaald dan bereken ik de cdf (die noem ik vCdf). Dit doe ik aan de hand van de functie scipy.integrate.cumtrapz.
<break>
In regel 20 heb ik dan de cdf van $Y_t$. In regel 24/25 bereken ik de integraal van de gehele pdf. Uit de integraal moet er 1 uitkomen. In regel 35 check ik of de integraal van mijn gehele pdf gelijk is aan 1. Want als dat niet zo is dan weet ik dat mijn cdf niet betrouwbaar zijn. Als het goed is elke integraal heel dichtbij 1 en in ieder geval kleiner dan 1.
<break>
Uiteindelijk wil ik voor de PIT de cdf weten die hoort bij $ P(Y_t<y_t)$. Dit doe ik dan aan de hand van dummy variables. Als de waarde van mijn gridpoint op positie i $\leq y_t$ dan krijgt dat punt een waarde 1 en anders een 0. Ik tel dan de som van de dummy variables. Dan weet ik hoeveel gridpoints een waarde 1 hebben. Uiteindelijk doe ik -1 zodat ik de positie heb van de laatste gridpoint die een waarde heeft van 1. Uitendelijk return ik de bijbehorende cdf 
<break>
Dit proces voor ik voor elke $y_t$ uit met de bijbehorende parameters

In [8]:
def cdfSkew(vTheta, dReturn, dVariance):
    vGridPoints=np.arange(-20,20,0.001)
    dOmega=vTheta[0]
    dAlpha=vTheta[1]
    dBeta=vTheta[2]
    dNu = vTheta[3]
    dXi = vTheta[4]
    dM=gamma( (dNu-1)/2 ) / ( gamma(dNu/2) )*np.sqrt( (dNu-2)/np.pi )*( dXi-1/dXi )
    dS=np.sqrt((dXi**2+(1/dXi**2)-1)-dM**2)
    vCondition = (dS * (vGridPoints/np.sqrt(dVariance))) + dM
    vI1 = vCondition>=0
    vI1 = vI1 + np.zeros(len(vCondition))
    vI2 = vCondition<0
    vI2 = -1*(vI2 + np.zeros(len(vCondition)))
    vI  = vI1+vI2    
   
    vPdf = gamma((dNu+1)/2)/gamma(dNu/2) * ((dNu-2)*np.pi*dVariance)**(-1/2) * dS * ( 2 / (dXi+1/dXi) ) * \
           (1 + ( (dS * ( ( vGridPoints/np.sqrt( dVariance ) ) ) + dM)**2/(dNu-2))*dXi**(-2*vI))**(-(dNu+1)/2)
    
    vCdf = integrate.cumtrapz(x=vGridPoints, y=vPdf, initial = 0)
   
    dProbability=integrate.simps(x=vGridPoints, y = vPdf)
    
    if dProbability<0.99 or dProbability>1 :
        print("does not integrate to 1")
    
    vDummy=vGridPoints<dReturn
    vDummy=vDummy+np.zeros(len(vCdf))
    dIndex=np.sum(vDummy)-1
    return vCdf[int(dIndex)]            

In de cel hieronder verkrijg ik de probabilities van de PIT. Ik zal later de inverse nemen van de probabilities, zodat ik de kwantielen later in de copula log-likelihood kan stoppen. 

In [9]:
vU1=np.zeros((iT,1))
vU2=np.zeros((iT,1))
for i in range(iT):
    vU1[i]=cdfSkew(parameter1_Skew,np.array(mInSampleReturns)[i,0],mSigma2[0,i])
    vU2[i]=cdfSkew(parameter2_Skew,np.array(mInSampleReturns)[i,1],mSigma2[1,i])   
    

In [10]:
print(np.min(vU1))
print(np.max(vU1))
print(np.min(vU2))
print(np.max(vU2))

0.00153362761199
0.999464619734
0.00187266972411
0.999012528518


### Berekenen van de CDF
Om de kwantielen te bepalen moeten we eerst de CDF opstellen van ons univariate copula met skewed student-t verdeling. De verdeling is voor elk tijdstip hetzelfde. Vandaar dat we eerst de CDF opstellen en die voor elk tijdstip t gaan gebruiken om de kwantielen te bepalen. De pdf met mean=0 en variance=1 van de univariate copula met skewed student-t verdeling ziet er als volgt uit:

$$f(x|\nu,\xi_i)=\frac{c\mathbb{K}_{\frac{1+\nu}{2}}\left(\sqrt{(\nu+x^2)\xi_i^2}\right)\exp(x\xi_i)}{\left(\sqrt{(\nu+x^2)\xi_i^2}\right)^{-\frac{1+\nu}{2}}\left(1+\frac{x^2}{\nu}\right)^\frac{1+\nu}{2}}$$

- c is als volgt gedefinieerd:
$$c=\frac{2^{\frac{2-(\nu+1)}{2}}}{\Gamma\left(\frac{\nu}{2}\right)(\pi\nu)^{\frac{1}{2}}}$$

- $\mathbb{K}_{\frac{1+\nu}{2}}(.)$ is de modified Bessel function of second kind
<break>

Ik maak weer een grid aan van -20 tot 20 en op elke grid bereken ik de bijbehorende pdf waarde. Uiteindelijk maak ik gebruik van de functie integrate.cumtrapz om van pdf naar cdf te gaan. 

In [11]:
def GetCDF(dNu,dXi):
    vGridPoints = np.arange(-20,20,0.001)
    iDimension = 1
    dC = 2**( ( 2- ( dNu+iDimension ) ) / 2) / (gamma(dNu/2) * np.sqrt(np.pi*dNu) )
    dSuperscript = np.sqrt((dNu + vGridPoints**2)*dXi**2)
    vPdf = (dC * kv((dNu+iDimension)/2,dSuperscript) * np.exp(vGridPoints*dXi)) / (dSuperscript**(- ( iDimension+dNu )/2 ) \
            * (1+ vGridPoints**2/dNu)**((iDimension+dNu)/2))

    vCdf = integrate.cumtrapz(x=vGridPoints, y=vPdf, initial = 0)
    return vCdf

### Het verkrijgen van de kwantielen.

In de copula log-likelihood moeten we de kwantielen van de PIT gebruiken. Hiervoor moeten we de inverse CDF gebruiken van de univariate copula skewed student-t met mean=0 and variance=1. In de functie hieronder roep ik de cdf van de univariate copula skewed student-t verdeling op met mean 0 and std 0. Aan de hand van de gegeven $\nu$ en $\xi$ gaan we de kwantielen bepalen van de cdf's die we in de cel hierboven hebben opgesteld.
<break>
Het verkrijgen van de kwantielen gaat op een soortgelijke manier als het verkrijgen van de probabilities. Dit gaan we ook bepalen met dummy variables. Ik kijk dan welke cdf's in de vector van vCdf kleiner zijn dan dU. Die krijgen dan een waarde 1 en de rest 0. Uiteindelijk neem ik de som zodat ik weet hoeveel van die cdf's een dummy variable 1 hebben gekregen. Om de index te weten van de laatste cdf met een dummy variable 1 moet ik dus np.sum(vDummy)-1 doen. Uiteindelijk return ik de de bijbehorende gridpoint. Dit doe ik in een for loop voor alle tijdstippen.   


In [12]:
def InverseCDF(dNu, dXi, vU):
    vGridPoints = np.arange(-20,20,0.001)
    vCdf=GetCDF(dNu,dXi)

    vX=np.zeros(len(vU))
    for i in range(len(vU)):
        dU = vU[i]
        vDummy=vCdf<dU
        vDummy=vDummy+np.zeros(len(vDummy))
        dIndex=np.sum(vDummy)-1
        if int(dIndex)==0:
            vX[i]=vGridPoints[0]
        
        elif int(dIndex) + 1 == len(vGridPoints):
            vX[i]= vGridPoints[int(dIndex)]
            
        else:
            vX[i]=vGridPoints[int(dIndex)] +( vGridPoints[int(dIndex)+1] - vGridPoints[int(dIndex)] )  * \
                  (dU - vCdf [int(dIndex)] ) / ( vCdf [int(dIndex)+1] - vCdf [int(dIndex)] ) 
    return vX


### Opstellen van de log-likelihood
In de cel hieronder stel ik de log-likelihood op. De log-likelihood ziet er als volgt uit:
$$\log(c)+\log\left(\mathbb{K}_{\frac{d+\nu}{2}}\left(\sqrt{(\nu+x'\Sigma^{-1}x)\gamma'\Sigma^{-1}\gamma}\right)\right)+x'\Sigma^{-1}\gamma + \frac{d+\nu}{2}\log\left(\sqrt{(\nu+x'\Sigma^{-1}x)\gamma'\Sigma^{-1}\gamma}\right)-\frac{d+\nu}{2}\log\left(1+\frac{x'\Sigma^{-1}x}{\nu}\right)-\log(f(x_{1t}|\nu,\xi_1))-\log(f(x_{2t}|\nu,\xi_2))$$

- $\mathbb{K}_{\frac{d+\nu}{2}}(.)$ is de modified Bessel function of the second kind met index $\frac{d+\nu}{2}$
- x is gedefinieerd als:
$$x=\begin{pmatrix} x_{1t}\\ x_{2t}\end{pmatrix}$$
waarbij $x_{it}$ een kwantiel is die verkregen is door de inverse cdf: $x_{it}=Skwt^{-1}_{\nu,\xi_i}(u_{it})$ voor i=1,2 en $t=1\dots n$. Waarbij $u_{it}$ de kansen van de tijdreeksen zijn die verkregen zijn door PIT.

- $\gamma$ is gedefinieerd als:
$$\gamma=\begin{pmatrix} \xi_{1}\\ \xi_{2}\end{pmatrix}$$

- In dit geval is $\Sigma=R$ en $R$ is als volgt gedefinieerd:

$$R=\begin{pmatrix} 1 & \rho \\ \rho & 1 \end{pmatrix}$$

- c is gedefinieerd als:
$$c=\frac{2^{\frac{2-(\nu+d)}{2}}}{\Gamma\left(\frac{\nu}{2}\right)(\pi\nu)^{\frac{d}{2}}\left|\Sigma\right|^{0.5}} $$

- d is de dimensie van de copula. In dit geval 2

- $f(x_{it}|\nu,\xi_i)$ is de univariate copula met skewed student-t verdeling die gedefinieerd is boven GetCDF en onder deze cel
<br>
<br>
De parameters die we moeten optimaliseren zijn $\rho,\nu,\xi_1,\xi_2$

### Definitie van de $f(x_{it}|\nu,\xi_i)$
Dit is de pdf die nodig is om in de log-likeihood te stoppen. In andere woorden dit is de $f(x_{it}|\nu,\xi_i)$ in de cel hierboven. De functie $f(x_{it}|\nu,\xi_i)$ is als volgt gedefnieerd:
$$f(x|\nu,\xi_i)=\frac{c\mathbb{K}_{\frac{1+\nu}{2}}\left(\sqrt{(\nu+x^2)\xi_i^2}\right)\exp(x\xi_i)}{\left(\sqrt{(\nu+x^2)\xi_i^2}\right)^{-\frac{1+\nu}{2}}\left(1+\frac{x^2}{\nu}\right)^\frac{1+\nu}{2}}$$

- c is als volgt gedefinieerd:
$$c=\frac{2^{\frac{2-(\nu+1)}{2}}}{\Gamma\left(\frac{\nu}{2}\right)(\pi\nu)^{\frac{1}{2}}}$$

- $\mathbb{K}_{\frac{1+\nu}{2}}(.)$ is de modified Bessel function of second kind
<break>

In [13]:
def getPDF(dX,dNu,dXi):
    iDimension = 1
    dC = 2**( ( 2- ( dNu+iDimension ) ) / 2) / (gamma(dNu/2) * np.sqrt(np.pi*dNu) )
    dSuperscript = np.sqrt((dNu + dX**2)*dXi**2)
    dPdf = (dC * kv((dNu+iDimension)/2,dSuperscript) * np.exp(dX*dXi)) / (dSuperscript**(- ( iDimension+dNu )/2 ) \
           * (1+ dX**2/dNu)**((iDimension+dNu)/2))
    return dPdf

Hier wordt de log-likelihood gedefinieerd die boven is beschreven.

In [17]:
def Parameters_Copula_Skew(vU1,vU2):
    dRho12 = 0.91677754
    dNu = 7.40781997
    dXi1 = 0.00001
    dXi2 = 0.00001
    vTheta = np.array([dRho12, dNu, dXi1, dXi2])
    
    def LL_Copula_Skew(vTheta,vU1,vU2):

        dRho12 = vTheta[0]
        dNu = vTheta[1]
        dXi1 = vTheta[2]
        dXi2 = vTheta[3]
        vGamma=np.array([[dXi1],[dXi2]])
        iT = len(vU1)
        #mQuantile= st.t.ppf(mUt, dNu, loc=np.zeros(mUt.shape), scale=np.ones(mUt.shape))
        iDimension = 2
        mR = np.ones((iDimension,iDimension))
        mR[1,0] = dRho12
        mR[0,1] = dRho12
        mInv_R = inv(mR)
        dC = 2**( (2- ( dNu+iDimension ) ) / 2 ) / ( gamma( dNu/2 ) * ( np.pi*dNu )**( iDimension/2 ) * np.sqrt(det(mR)) ) 
        vX1=InverseCDF(dNu, dXi1, vU1)
        vX2=InverseCDF(dNu, dXi2, vU2)
        dSum=0
        
        for t in range(0,iT):
            vX = np.zeros( ( iDimension,1 ) )
            vX[0] = vX1[t]
            vX[1] = vX2[t]
            dPdf1 = getPDF(vX[0], dNu, dXi1)
            dPdf2 = getPDF(vX[1], dNu, dXi2)
            dSuperscript = np.sqrt( (dNu + vX.T @ mInv_R @ vX ) @ vGamma.T @ mInv_R @ vGamma)
            dLogLikelihood = np.log(dC)+np.log(kv((dNu+iDimension)/2,dSuperscript)) + (vX.T @ mInv_R @ vGamma) + \
                         (dNu+iDimension)/2 * np.log(dSuperscript) - (dNu+iDimension)/2 * np.log(1+ (vX.T @ mInv_R @ vX)/dNu) \
                         -np.log(dPdf1) - np.log(dPdf2)   
            dSum += np.asscalar(dLogLikelihood)
        #vAppel.append([vTheta,-dSum])
        #print(dLogLikelihood)
        #print(vTheta.T)
        
        return -dSum
        
    def Optimizer(vU1, vU2, initials, function, bnds):
        result = minimize(function, initials, args=(vU1,vU2), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    #vAppel=[]
    bounds = ((-0.9999999, 0.9999999),(2.1,50), (-2,2), (-2,2))
    result=Optimizer(vU1, vU2, vTheta, LL_Copula_Skew, bounds)
    return result.x, -result.fun, result.success

In [15]:
import timeit

In [18]:
start = timeit.default_timer()
(vTheta,LL_Copula_Skew,success_Copula_Skew)=Parameters_Copula_Skew(vU1,vU2)
stop = timeit.default_timer()
execution_time = stop - start
print(vTheta)
print(LL_Copula_Skew)
print(success_Copula_Skew)
print(execution_time)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -1492.514546530313
            Iterations: 15
            Function evaluations: 122
            Gradient evaluations: 15
[ 0.92326162  5.14010071 -0.05907519 -0.06048458]
1492.514546530313
True
99.80720170000001
